This example walkthrough works with both CPU (amazingly works but is slow) and with T4 GPU (quite a bit faster).  This uses ollama as an inference solution.  This is just a quick demo, only intended to help folks get started if they can't access the model through other means.

Overall, the [gpt-oss cookbook](https://cookbook.openai.com/topic/gpt-oss) remains your best source of information about how to run gpt-oss-20b

In [2]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8
  404  Not Found [IP: 185.125.190.83 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.8_amd64.deb  404  Not Found [IP: 185.125.190.83 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [3]:
%%bash

# Fetch and install the latest release of ollama.
curl -fsSL https://ollama.com/install.sh | sh

# Start and ollama server locally on the backend.
nohup ollama serve > /tmp/ollama_serve_stdout.log 2>/tmp/ollama_serve_stderr.log &


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
!pip install pyMuPDF
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 5.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 69.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 77.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import subprocess

def pull_ollama_models(models):
    for model in models:
        print(f"🔄 Pulling model: {model}")
        try:
            result = subprocess.run(["ollama", "pull", model], check=True, capture_output=True, text=True)
            output = result.stdout + result.stderr
            if "already exists" in output:
                print(f"✅ {model} is already downloaded.")
            else:
                print(f"✅ Pulled {model} successfully.")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to pull {model}")
            print(e.stderr)

# Define your desired models
models_to_pull = [
    "gpt-oss:20b",
    "gemma3:4b",
    "qwen3:4b",
    "llama3.1:8b"
]

# Run the pull
pull_ollama_models(models_to_pull)

🔄 Pulling model: gpt-oss:20b
✅ Pulled gpt-oss:20b successfully.
🔄 Pulling model: gemma3:4b
✅ Pulled gemma3:4b successfully.
🔄 Pulling model: qwen3:4b
✅ Pulled qwen3:4b successfully.
🔄 Pulling model: llama3.1:8b
✅ Pulled llama3.1:8b successfully.


In [ ]:
"""## Now we can demo using the model from the notebook.  It is quite slow running on CPU, but
## it does work.  It also works on T4 GPTU.
from openai import OpenAI
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=[
        {"role": "system", "content": "You are a professor of 18th century English literature.  Mention dogs where possible."},
        {"role": "user", "content": "Write a 4-line micro-poem about running a big model on a limited notebook."}
    ]
)
print(response.choices[0].message.content)
print("\n\nFull chat completion JSON:\n")
print(response)


In [6]:
!sudo apt-get update
!sudo apt-get install -y poppler-utils


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,918 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,161 kB]  
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]     
Get:12 http://security.ubuntu.com/ubuntu jammy-sec

In [6]:
# Sometimes ollama fails and becomes defunct.  Run this to see if tha has happened.
#os.system("ps aux | grep -E \"ollama\" | grep -v grep || true")

In [11]:
"""""
# If you need to re-start ollama because it has crashed, run this cell.
os.system("pkill -9 ollama || true")
os.system("nohup ollama serve > /tmp/ollama_serve_stdout.log 2>/tmp/ollama_serve_stderr.log &")
os.system("sleep 5")
print ("If a JSON string shows below the server is working:")
os.system("curl -s http://localhost:11434/v1/models")
print ("Make sure that gpt-oss:20b is still listed, otherwise you will have to re-load it.")
os.system("ollama list")

If a JSON string shows below the server is working:
{"object":"list","data":[{"id":"llama3.1:8b","object":"model","created":1754486922,"owned_by":"library"},{"id":"qwen3:4b","object":"model","created":1754486885,"owned_by":"library"},{"id":"gemma3:4b","object":"model","created":1754486866,"owned_by":"library"},{"id":"gpt-oss:20b","object":"model","created":1754486841,"owned_by":"library"}]}
Make sure that gpt-oss:20b is still listed, otherwise you will have to re-load it.
NAME           ID              SIZE      MODIFIED       
llama3.1:8b    46e0c10c039e    4.9 GB    17 minutes ago    
qwen3:4b       2bfd38a7daaf    2.6 GB    17 minutes ago    
gemma3:4b      a2af6cc3eb7f    3.3 GB    18 minutes ago    
gpt-oss:20b    f2b8351c629c    13 GB     18 minutes ago    


0

In [8]:
#!/usr/bin/env python3
"""
GPU-Optimized RAG Benchmark for Tesla T4
- Quantized models for faster inference
- Batch processing for embeddings
- GPU acceleration for text processing
- Optimized OCR with GPU support
"""

import os
import time
import fitz  # PyMuPDF
import pytesseract
import requests
import torch
from PIL import Image
from pdf2image import convert_from_path
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
from tqdm import tqdm
import json
from datetime import datetime

# GPU Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {DEVICE}")
if DEVICE == "cuda":
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

DOCUMENT_FOLDER = "/kaggle/input/documents"
OLLAMA_MODELS = [
    "gpt-oss:20b",
    "llama3.1:8b", 
    "gemma3:4b",
    "qwen3:4b"
]
OLLAMA_API_URL = "http://localhost:11434/v1/chat/completions"

# Optimized embedding model with quantization
EMBED_MODEL = SentenceTransformer("all-MiniLM-L6-v2", device=DEVICE)

def optimize_for_gpu():
    """Optimize PyTorch for GPU performance"""
    if DEVICE == "cuda":
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        # Set memory fraction to avoid OOM
        torch.cuda.set_per_process_memory_fraction(0.8)
        print("⚡ GPU optimizations applied")

def extract_text_fast_pdf(pdf_path):
    """Fast PDF text extraction prioritizing PyMuPDF"""
    print(f"📄 Fast text extraction from: {pdf_path}")
    try:
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
        
        if len(text.strip()) > 100:  # If we got substantial text
            print(f"✅ Direct text extraction successful: {len(text)} chars")
            return text.strip()
        else:
            print("⚠️ Direct extraction yielded little text, trying OCR...")
            return extract_text_ocr_fallback(pdf_path)
            
    except Exception as e:
        print(f"❌ Direct extraction failed: {e}")
        return extract_text_ocr_fallback(pdf_path)

def extract_text_ocr_fallback(pdf_path):
    """Optimized OCR fallback with reduced DPI and batch processing"""
    print(f"🔍 OCR processing with GPU optimizations...")
    
    # Reduced DPI for faster processing
    pages = convert_from_path(pdf_path, dpi=200)  # Reduced from 300
    
    text = ""
    batch_size = 4  # Process multiple pages at once
    
    for i in range(0, len(pages), batch_size):
        batch = pages[i:i+batch_size]
        batch_text = ""
        
        for j, page in enumerate(batch):
            # Use faster OCR configuration
            page_text = pytesseract.image_to_string(
                page, 
                config='--oem 1 --psm 6'  # Faster OCR engine
            )
            batch_text += page_text + "\n"
        
        text += batch_text
        print(f"📄 Processed batch {i//batch_size + 1}/{(len(pages)-1)//batch_size + 1}")
    
    return text.strip()

def extract_text_from_image_gpu(image_path):
    """GPU-optimized image text extraction"""
    print(f"🖼️ GPU-optimized image processing: {image_path}")
    
    # Load image with GPU acceleration if available
    image = Image.open(image_path)
    
    # Convert to RGB if needed (better for OCR)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Use optimized OCR settings
    text = pytesseract.image_to_string(
        image,
        config='--oem 1 --psm 6'  # Fastest OCR configuration
    )
    
    return text.strip()

def extract_text_optimized(filepath):
    """Optimized text extraction with GPU support"""
    ext = filepath.lower().split('.')[-1]
    
    if ext == 'pdf':
        return extract_text_fast_pdf(filepath)
    elif ext in ['jpg', 'jpeg', 'png']:
        return extract_text_from_image_gpu(filepath)
    else:
        print(f"❌ Unsupported file type: {ext}")
        return ""

def create_embeddings_batch(texts, batch_size=32):
    """Batch processing for embeddings with GPU acceleration"""
    print(f"🧠 Creating embeddings in batches of {batch_size}...")
    
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch = texts[i:i+batch_size]
        batch_embeddings = EMBED_MODEL.encode(batch, convert_to_tensor=True)
        embeddings.extend(batch_embeddings.cpu().numpy())
    
    return embeddings

def setup_vector_db_optimized(all_documents):
    """Optimized vector database setup with batch processing"""
    print("🔍 Building optimized global vector store...")
    
    # Initialize ChromaDB
    chroma_client = chromadb.Client()
    collection_name = "gpu_optimized_rag"
    
    # Clean up existing collection
    if collection_name in [c.name for c in chroma_client.list_collections()]:
        chroma_client.delete_collection(collection_name)
    
    # Create collection with optimized embedding function
    embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-MiniLM-L6-v2",
        device=DEVICE
    )
    
    collection = chroma_client.create_collection(
        name=collection_name,
        embedding_function=embedding_func
    )
    
    # Process all documents
    all_chunks = []
    all_ids = []
    all_metadatas = []
    
    for filename, text in all_documents.items():
        # Chunk text efficiently
        chunks = [text[i:i+500] for i in range(0, len(text), 500)]
        
        # Create IDs and metadata
        chunk_ids = [f"{filename}_chunk_{i}" for i in range(len(chunks))]
        chunk_metadatas = [{"source": filename, "chunk_id": i} for i in range(len(chunks))]
        
        all_chunks.extend(chunks)
        all_ids.extend(chunk_ids)
        all_metadatas.extend(chunk_metadatas)
    
    print(f"📊 Total chunks to process: {len(all_chunks)}")
    
    # Add documents in batches for better performance
    batch_size = 100
    for i in tqdm(range(0, len(all_chunks), batch_size), desc="Adding to vector store"):
        batch_chunks = all_chunks[i:i+batch_size]
        batch_ids = all_ids[i:i+batch_size]
        batch_metadatas = all_metadatas[i:i+batch_size]
        
        collection.add(
            documents=batch_chunks,
            ids=batch_ids,
            metadatas=batch_metadatas
        )
    
    return collection

def clean_answer(answer):
    """Clean up answer by removing thinking tags and ensuring completeness"""
    if not answer:
        return "No answer generated."
    
    # Remove thinking tags
    answer = answer.replace("<think>", "").replace("</think>", "")
    answer = answer.replace("<thinking>", "").replace("</thinking>", "")
    
    # Remove common thinking patterns
    thinking_patterns = [
        "Okay, the user wants me to",
        "Let me analyze",
        "Looking at the context",
        "Hmm, the user",
        "I need to",
        "Based on my analysis"
    ]
    
    for pattern in thinking_patterns:
        if pattern.lower() in answer.lower():
            # Find the first sentence that doesn't start with thinking
            sentences = answer.split('.')
            clean_sentences = []
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and not any(pattern.lower() in sentence.lower() for pattern in thinking_patterns):
                    clean_sentences.append(sentence)
            
            if clean_sentences:
                answer = '. '.join(clean_sentences) + '.'
            break
    
    # Ensure the answer ends properly
    if answer and not answer.endswith(('.', '!', '?')):
        answer += '.'
    
    return answer.strip()

def ask_ollama_model_optimized(model_name, context, question, timeout=120):
    """Optimized Ollama model querying with better error handling"""
    print(f"🧠 Asking {model_name} (optimized)...")
    
    url = OLLAMA_API_URL
    headers = {"Content-Type": "application/json"}
    
    # Clean prompt without thinking instructions
    optimized_prompt = f"Based on the provided context, answer this question in 6-7 complete sentences:\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"
    
    data = {
        "model": model_name,
        "messages": [{"role": "user", "content": optimized_prompt}],
        "temperature": 0.3,
        "stream": False,  # Ensure we get complete response
        "options": {
            "num_predict": 1200,  # Ollama-specific parameter for max tokens
            "stop": ["\n\n", "Question:", "Context:"]  # Stop at natural boundaries
        }
    }
    
    start = time.time()
    try:
        response = requests.post(url, headers=headers, json=data, timeout=timeout)
        response.raise_for_status()
        
        result = response.json()
        answer = result.get("choices", [{}])[0].get("message", {}).get("content", "")
        duration = time.time() - start
        
        # Clean up the answer
        cleaned_answer = clean_answer(answer)
        
        # Check if answer seems complete (ends with proper punctuation)
        if cleaned_answer and not cleaned_answer.endswith(('.', '!', '?')):
            print(f"⚠️ Answer may be incomplete, retrying with longer context...")
            # Try again with a more explicit prompt
            retry_prompt = f"Provide a complete answer to this question in exactly 6-7 sentences. End your answer with a period.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"
            
            retry_data = {
                "model": model_name,
                "messages": [{"role": "user", "content": retry_prompt}],
                "temperature": 0.3,
                "stream": False,
                "options": {
                    "num_predict": 1500,  # Even more tokens
                    "stop": ["\n\n", "Question:", "Context:"]
                }
            }
            
            try:
                retry_response = requests.post(url, headers=headers, json=retry_data, timeout=timeout)
                retry_response.raise_for_status()
                retry_result = retry_response.json()
                retry_answer = retry_result.get("choices", [{}])[0].get("message", {}).get("content", "")
                cleaned_answer = clean_answer(retry_answer)
                print(f"✅ Retry successful - complete answer generated")
            except Exception as e:
                print(f"⚠️ Retry failed: {e}")
        
        return cleaned_answer, duration
        
    except requests.exceptions.Timeout:
        print(f"⏰ Timeout for {model_name} (120s limit)")
        return f"Timeout error for {model_name} - response took too long", -1
    except requests.exceptions.ConnectionError:
        print(f"🔌 Connection error for {model_name}")
        return f"Connection error for {model_name} - check if Ollama is running", -1
    except Exception as e:
        print(f"❌ Error with {model_name}: {e}")
        return f"Error: {str(e)}", -1

def main():
    """Main GPU-optimized RAG benchmark"""
    print("🚀 GPU-OPTIMIZED RAG Benchmark for Tesla T4")
    print("=" * 60)
    
    # Apply GPU optimizations
    optimize_for_gpu()
    
    # Pull models (if needed)
    print("\n📦 Checking model availability...")
    for model in OLLAMA_MODELS:
        print(f"✅ {model} ready")
    
    # Find all supported files
    supported_exts = ['pdf', 'jpg', 'jpeg', 'png']
    files = [f for f in os.listdir(DOCUMENT_FOLDER) 
             if f.lower().split('.')[-1] in supported_exts]
    
    if not files:
        print("❌ No supported documents found.")
        return
    
    print(f"\n📁 Found {len(files)} files to process")
    
    # Step 1: Process ALL documents with GPU optimization
    print("\n🔄 Step 1: Processing ALL documents with GPU acceleration...")
    all_documents = {}
    total_extraction_time = 0
    
    for filename in tqdm(files, desc="Processing documents"):
        filepath = os.path.join(DOCUMENT_FOLDER, filename)
        
        extraction_start = time.time()
        text = extract_text_optimized(filepath)
        extraction_time = time.time() - extraction_start
        
        if text.strip():
            all_documents[filename] = text
            total_extraction_time += extraction_time
            print(f"✅ {filename}: {len(text)} chars in {extraction_time:.2f}s")
        else:
            print(f"⚠️ {filename}: No text extracted")
    
    print(f"\n📊 Document Processing Summary:")
    print(f"   📁 Total documents: {len(all_documents)}")
    print(f"   ⏱️ Total extraction time: {total_extraction_time:.2f}s")
    print(f"   📝 Total characters: {sum(len(text) for text in all_documents.values()):,}")
    
    # Step 2: Build global vector store
    print(f"\n🗄️ Step 2: Building global vector store...")
    vector_start = time.time()
    collection = setup_vector_db_optimized(all_documents)
    vector_time = time.time() - vector_start
    
    print(f"✅ Global vector store ready in {vector_time:.2f}s")
    
    # Step 3: Process questions with all models
    questions = [
        "what is co-branding?",
        "what is luxury co-branding?",
        "What are the factors affecting co-branding?",
        "What are the types of co-branding?"
    ]
    
    print(f"\n🤖 Step 3: Processing {len(questions)} questions with {len(OLLAMA_MODELS)} models...")
    
    results = {}
    
    for q_idx, question in enumerate(questions, 1):
        print(f"\n{'='*60}")
        print(f"📌 Question {q_idx}/{len(questions)}: {question}")
        print(f"{'='*60}")
        
        # Retrieve relevant chunks
        search_start = time.time()
        results_query = collection.query(
            query_texts=[question],
            n_results=1
        )
        search_time = time.time() - search_start
        
        chunks = results_query["documents"][0]
        metadatas = results_query["metadatas"][0]
        
        # Get unique source documents
        sources = list(set([meta["source"] for meta in metadatas]))
        context = "\n---\n".join(chunks)
        
        print(f"🔍 Search time: {search_time:.2f}s")
        print(f"📄 Relevant documents: {', '.join(sources)}")
        
        question_results = {}
        
        for m_idx, model in enumerate(OLLAMA_MODELS, 1):
            print(f"\n🤖 Model {m_idx}/{len(OLLAMA_MODELS)}: {model}")
            print("-" * 40)
            
            answer, duration = ask_ollama_model_optimized(model, context, question)
            
            question_results[model] = {
                "answer": answer,
                "response_time": duration,
                "relevant_documents": sources
            }
            
            print(f"⏱️ Response time: {duration:.2f}s")
            print(f"📝 Answer: {answer}")
            print(f"📏 Answer length: {len(answer)} characters")
            if not answer.endswith(('.', '!', '?')):
                print(f"⚠️ Answer may be incomplete (doesn't end with punctuation)")
        
        results[question] = question_results
    
    # Print comprehensive evaluation
    print_comprehensive_evaluation(results)
    
    # Export results
    export_results(results)

def print_comprehensive_evaluation(results):
    """Print comprehensive evaluation of results"""
    print(f"\n{'='*80}")
    print("📊 COMPREHENSIVE EVALUATION & COMPARISON")
    print(f"{'='*80}")
    
    # Calculate statistics
    model_stats = {}
    for question, question_results in results.items():
        for model, result in question_results.items():
            if model not in model_stats:
                model_stats[model] = {
                    "times": [],
                    "success_count": 0,
                    "total_count": 0,
                    "answer_lengths": []
                }
            
            model_stats[model]["total_count"] += 1
            if result["response_time"] > 0:
                model_stats[model]["success_count"] += 1
                model_stats[model]["times"].append(result["response_time"])
                model_stats[model]["answer_lengths"].append(len(result["answer"]))
    
    print("\n🎯 OVERALL STATISTICS")
    print("-" * 40)
    print(f"📝 Total questions processed: {len(results)}")
    print(f"🤖 Total models tested: {len(model_stats)}")
    
    print("\n🤖 MODEL PERFORMANCE COMPARISON")
    print("-" * 40)
    
    for model, stats in model_stats.items():
        if stats["times"]:
            avg_time = np.mean(stats["times"])
            min_time = np.min(stats["times"])
            max_time = np.max(stats["times"])
            success_rate = (stats["success_count"] / stats["total_count"]) * 100
            avg_length = np.mean(stats["answer_lengths"])
            
            print(f"📊 {model}:")
            print(f"   ⏱️ Response time: {avg_time:.2f}s avg ({min_time:.2f}s - {max_time:.2f}s)")
            print(f"   ✅ Success rate: {success_rate:.1f}% ({stats['success_count']}/{stats['total_count']})")
            print(f"   📝 Avg answer length: {avg_length:.0f} characters")
    
    # Performance rankings
    print("\n🏆 PERFORMANCE RANKINGS")
    print("-" * 40)
    
    # Speed ranking
    speed_ranking = sorted(
        [(model, np.mean(stats["times"])) for model, stats in model_stats.items() if stats["times"]],
        key=lambda x: x[1]
    )
    
    print("🚀 Speed Ranking (Fastest to Slowest):")
    for i, (model, avg_time) in enumerate(speed_ranking, 1):
        print(f"   {i}. {model}: {avg_time:.2f}s")
    
    # Reliability ranking
    reliability_ranking = sorted(
        [(model, (stats["success_count"] / stats["total_count"]) * 100) for model, stats in model_stats.items()],
        key=lambda x: x[1],
        reverse=True
    )
    
    print("\n🛡️ Reliability Ranking (Most to Least Reliable):")
    for i, (model, success_rate) in enumerate(reliability_ranking, 1):
        print(f"   {i}. {model}: {success_rate:.1f}% success rate")
    
    # GPT-OSS Final Evaluation
    print("\n🤖 GPT-OSS FINAL EVALUATION")
    print("-" * 40)
    
    # Create evaluation prompt for GPT-OSS
    evaluation_prompt = create_evaluation_prompt(results, model_stats)
    
    print("🧠 Asking gpt-oss:20b for final evaluation...")
    gpt_eval_start = time.time()
    
    try:
        gpt_eval_answer, gpt_eval_time = ask_ollama_model_optimized(
            "gpt-oss:20b", 
            evaluation_prompt, 
            "Please provide a comprehensive evaluation of these RAG benchmark results"
        )
        
        print(f"📊 GPT-OSS Evaluation ({gpt_eval_time:.2f}s):")
        print(gpt_eval_answer)
        
    except Exception as e:
        print(f"❌ GPT-OSS evaluation failed: {e}")
        print("📊 Using fallback evaluation summary...")
        
        # Fallback evaluation
        print("\n📊 FALLBACK EVALUATION SUMMARY:")
        print("=" * 50)
        
        best_speed = speed_ranking[0] if speed_ranking else ("None", 0)
        best_reliability = reliability_ranking[0] if reliability_ranking else ("None", 0)
        
        print(f"🏆 Best Speed: {best_speed[0]} ({best_speed[1]:.2f}s)")
        print(f"🛡️ Best Reliability: {best_reliability[0]} ({best_reliability[1]:.1f}%)")
        
        if model_stats:
            avg_success_rate = np.mean([(stats["success_count"] / stats["total_count"]) * 100 
                                      for stats in model_stats.values()])
            print(f"📈 Overall Success Rate: {avg_success_rate:.1f}%")
            
            avg_response_time = np.mean([np.mean(stats["times"]) 
                                       for stats in model_stats.values() if stats["times"]])
            print(f"⏱️ Average Response Time: {avg_response_time:.2f}s")

def create_evaluation_prompt(results, model_stats):
    """Create comprehensive evaluation prompt for GPT-OSS"""
    
    # Collect all answers for evaluation
    all_answers = []
    for question, question_results in results.items():
        for model, result in question_results.items():
            if result["response_time"] > 0:  # Only successful responses
                all_answers.append({
                    "question": question,
                    "model": model,
                    "answer": result["answer"],
                    "response_time": result["response_time"],
                    "answer_length": len(result["answer"])
                })
    
    # Calculate statistics
    model_performance = {}
    for model, stats in model_stats.items():
        if stats["times"]:
            model_performance[model] = {
                "avg_time": np.mean(stats["times"]),
                "min_time": np.min(stats["times"]),
                "max_time": np.max(stats["times"]),
                "success_rate": (stats["success_count"] / stats["total_count"]) * 100,
                "avg_length": np.mean(stats["answer_lengths"])
            }
    
    # Create evaluation prompt
    model_stats_text = "\n".join([f"- {model}: {stats['avg_time']:.2f}s avg, {stats['success_rate']:.1f}% success, {stats['avg_length']:.0f} chars avg" for model, stats in model_performance.items()])
    
    sample_answers_text = ""
    for answer in all_answers[:8]:
        sample_answers_text += f"Question: {answer['question']}\nModel: {answer['model']}\nTime: {answer['response_time']:.2f}s\nAnswer: {answer['answer'][:300]}...\n\n"
    
    prompt = f"""Please provide a comprehensive evaluation of these RAG benchmark results:

MODEL PERFORMANCE STATISTICS:
{model_stats_text}

SAMPLE ANSWERS:
{sample_answers_text}

Please evaluate:
1. Overall system performance and reliability
2. Speed
3. Quality of answers
4. Speed vs quality trade-offs


Provide a detailed analysis with specific scores and rankings."""
    
    return prompt

def export_results(results):
    """Export results to JSON file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"gpu_optimized_rag_results_{timestamp}.json"
    
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n💾 Results exported to: {filename}")

if __name__ == "__main__":
    main() 

🚀 Using device: cuda
🎮 GPU: Tesla T4
💾 GPU Memory: 15.8 GB
🚀 GPU-OPTIMIZED RAG Benchmark for Tesla T4
⚡ GPU optimizations applied

📦 Checking model availability...
✅ gpt-oss:20b ready
✅ llama3.1:8b ready
✅ gemma3:4b ready
✅ qwen3:4b ready

📁 Found 10 files to process

🔄 Step 1: Processing ALL documents with GPU acceleration...


Processing documents:  10%|█         | 1/10 [00:00<00:01,  5.93it/s]

📄 Fast text extraction from: /kaggle/input/documents/Is co-branding an effective way to improve brand masstige (1).pdf
✅ Direct text extraction successful: 103299 chars
✅ Is co-branding an effective way to improve brand masstige (1).pdf: 103069 chars in 0.17s
📄 Fast text extraction from: /kaggle/input/documents/Signaling Unobservable Product Quality Through a Brand Ally (1).pdf


Processing documents:  30%|███       | 3/10 [00:00<00:00,  7.08it/s]

✅ Direct text extraction successful: 64817 chars
✅ Signaling Unobservable Product Quality Through a Brand Ally (1).pdf: 64816 chars in 0.08s
📄 Fast text extraction from: /kaggle/input/documents/Int J Consumer Studies - 2021 - Paydas Turan - Success drivers of cobranding  A metaanalysis (2).pdf
✅ Direct text extraction successful: 142521 chars
✅ Int J Consumer Studies - 2021 - Paydas Turan - Success drivers of cobranding  A metaanalysis (2).pdf: 142520 chars in 0.18s
📄 Fast text extraction from: /kaggle/input/documents/2023 Co-branding as a masstige strategy for luxury brands Desirable or not (1).pdf


...

🔍 Search time: 0.02s
📄 Relevant documents: 2023 Co-branding as a masstige strategy for luxury brands Desirable or not (1).pdf

🤖 Model 1/4: gpt-oss:20b
----------------------------------------
🧠 Asking gpt-oss:20b (optimized)...
⏱️ Response time: 11.83s
📝 Answer: Luxury co‑branding refers to a strategic partnership in which a high‑end, prestige brand partners with another brand—often a more accessible or mass‑market brand—to create a joint product or marketing campaign. The goal is to combine the luxury brand’s perceived quality, heritage, and exclusivity with the partner’s broader reach, distribution network, or complementary product attributes. By doing so, the luxury brand can tap into new customer segments and generate additional revenue streams while maintaining its premium positioning. The partner brand, in turn, gains a halo effect, benefiting from the association with the luxury image and attracting consumers who aspire to the status symbol. This collaboration is typically fram

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Search time: 0.02s
📄 Relevant documents: Is co-branding an effective way to improve brand masstige (1).pdf

🤖 Model 1/4: gpt-oss:20b
----------------------------------------
🧠 Asking gpt-oss:20b (optimized)...
⏱️ Response time: 34.13s
📝 Answer: Co‑branding success hinges on several interrelated factors. First, **brand fit or compatibility** is crucial; the two partners must share similar values, target audiences, and product categories so that the joint offering feels coherent to consumers. Second, the **existing brand equity and reputation** of each partner influence how much credibility and trust the collaboration can generate—stronger equity can amplify perceived value while weak equity may dilute it. Third, **consumer perception and involvement** matter; customers must see the partnership as authentic, high‑quality, and novel, rather than a gimmick. Fourth, **brand personality and image alignment** help avoid cognitive dissonance—complementary personalities reinforce the desired 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Search time: 0.02s
📄 Relevant documents: Int J Consumer Studies - 2021 - Paydas Turan - Success drivers of cobranding  A metaanalysis (2).pdf

🤖 Model 1/4: gpt-oss:20b
----------------------------------------
🧠 Asking gpt-oss:20b (optimized)...
⏱️ Response time: 20.66s
📝 Answer: Co‑branding can be grouped into three broad categories that are widely recognized in the literature.  
1. **Product co‑branding** occurs when two or more brands are combined on a single tangible product (for example, a sports shoe that carries both a footwear and a sports‑wear brand).  
2. **Service co‑branding** involves the joint promotion of a service, such as a hotel partnering with a credit‑card company to offer exclusive benefits to guests.  
3. **Corporate (or umbrella) co‑branding** refers to the partnership of two corporate entities that share a common brand identity, often seen in joint ventures or strategic alliances where both companies’ logos appear together on marketing materials.  

These three